In [1]:
from pymodulon.core import IcaData
import os
from os import path
import pandas as pd
import re
from Bio.KEGG import REST
from tqdm.notebook import tqdm

# Creating the ica_data object

In [56]:
processed = path.join("../data")

ica_data = IcaData(path.join(processed,"interim/ica_runs/120/S.csv"),
                   path.join(processed,"interim/ica_runs/120/A.csv"),
                   X=path.join(processed,"interim/log_tpm_norm.csv"),
                   sample_table=path.join(processed,"interim/metadata_qc.csv"), 
                   trn = path.join(processed,"external/AB5075_trn.csv"),
                   gene_table=path.join(processed,"external/gene_info.csv"),
                   imodulon_table=path.join(processed,"processed_data/iModulon_table.csv"))

# Incorporating iModulon characterization parameters

In [57]:
comp_names = list(pd.read_csv(path.join(processed,'processed_data/iModulon_table.csv'),index_col=1).index)

rename = {}

for old,new in zip(range(0,49),comp_names):
    rename.update({old:new})

ica_data.rename_imodulons(rename)

ica_data.change_threshold('Fur-1', 0.05)
ica_data.change_threshold('GigA-GigB KO', 0.2)
ica_data.change_threshold('R7 mutant', 0.057)
ica_data.change_threshold('cyd', 0.125)
ica_data.change_threshold('AMR', 0.08)

ica_data.A.loc['ABUW_1645'] = ica_data.A.loc['ABUW_1645'].multiply(-1, axis=0)
ica_data.M['ABUW_1645'] = ica_data.M['ABUW_1645'].multiply(-1)

# Incorporate TRN

In [58]:
ica_data.compute_trn_enrichment(save=True)

,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,Fur-1,Fur,1.501639e-20,7.508194e-20,0.365854,0.348837,0.357143,15.0,43.0,41.0,1.0
1,BfmR-BfmS,BfmR-BfmS,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000,6.0,6.0,6.0,1.0
2,BfmR-BfmS,AbaM,2.714043e-11,4.071064e-11,1.000000,0.085714,0.157895,6.0,70.0,6.0,1.0
3,Fur-2,Fur,3.853864e-26,1.541546e-25,0.365385,0.441860,0.400000,19.0,43.0,52.0,1.0
4,R7 mutant,AbaM,5.683560e-08,5.683560e-08,0.169492,0.142857,0.155039,10.0,70.0,59.0,1.0
5,ABUW_1645,ABUW_1645,2.987707e-16,1.195083e-15,0.583333,0.120690,0.200000,14.0,116.0,24.0,1.0
6,Prophage,Phage region 3,5.566916e-23,2.783458e-22,0.313725,0.484848,0.380952,16.0,33.0,51.0,1.0
7,Prophage,Phage region 5,1.576523e-18,3.941307e-18,0.294118,0.333333,0.312500,15.0,45.0,51.0,1.0
8,Prophage,Phage region 2,1.182879e-13,1.971464e-13,0.294118,0.168539,0.214286,15.0,89.0,51.0,1.0
9,Unc-1,Phage region 1,3.743553e-08,2.246132e-07,0.173913,0.173913,0.173913,8.0,46.0,46.0,1.0


# Annotating the gene_table

In [59]:
from pymodulon.gene_util import *

## GFF files 

In [60]:
#Import the gff files for AB5075 and plasmids
gff_AB5075 = '../data/external/sequence_files/AB5075.gff3'
gff_p1AB5075 = '../data/external/sequence_files/p1Ab5075.gff3'
gff_p2AB5075 = '../data/external/sequence_files/p2Ab5075.gff3'
gff_p3AB5075 = '../data/external/sequence_files/p3Ab5075.gff3'

In [61]:
#Convert gff files to dataframes
DF_annot = gff2pandas(gff_AB5075,index='locus_tag')
DF_annotp1 = gff2pandas(gff_p1AB5075,index='locus_tag')
DF_annotp2 = gff2pandas(gff_p2AB5075,index='locus_tag')
DF_annotp3 = gff2pandas(gff_p3AB5075,index='locus_tag')

#combine all gff dataframes into one dataframe
DF_annot = DF_annot.append(DF_annotp1)
DF_annot = DF_annot.append(DF_annotp2)
DF_annot = DF_annot.append(DF_annotp3)

#Remove the uneccessary info
keep_cols = ['accession','start','end','strand','gene_name','old_locus_tag','gene_product','ncbi_protein']
DF_annot = DF_annot[keep_cols]

DF_annot.head()

,accession,start,end,strand,gene_name,old_locus_tag,gene_product,ncbi_protein
locus_tag,,,,,,,,
ABUW_0001,CP008706.1,95,1492,+,dnaA,None,chromosomal replication initiator protein DnaA,AKA29801.1
ABUW_0002,CP008706.1,1590,2738,+,dnaN,None,DNA polymerase III%2C beta subunit,AKA29802.1
ABUW_0003,CP008706.1,2753,3835,+,recF,None,DNA replication%2C recombination and repair pr...,AKA29803.1
ABUW_0004,CP008706.1,3888,6356,+,gyrB,None,DNA gyrase%2C B subunit,AKA29804.1
ABUW_0005,CP008706.1,6394,6786,+,None,None,cytochrome b562,AKA29805.1


In [62]:
ica_data.gene_table = ica_data.gene_table.sort_index(ascending=True)

In [63]:
ica_data.gene_table['accession'] = DF_annot['accession']
ica_data.gene_table['ncbi_protein'] = DF_annot['ncbi_protein']
ica_data.gene_table.sort_index(ascending=True)

,gene_name,old_locus_tag,start,end,strand,gene_product,COG,uniprot,regulator,accession,ncbi_protein
ABUW_0001,dnaA,NaN,95,1492,+,chromosomal replication initiator protein DnaA,"Replication, recombination and repair",NaN,NaN,CP008706.1,AKA29801.1
ABUW_0002,dnaN,NaN,1590,2738,+,DNA polymerase III%2C beta subunit,"Replication, recombination and repair",NaN,NaN,CP008706.1,AKA29802.1
ABUW_0003,recF,NaN,2753,3835,+,DNA replication%2C recombination and repair pr...,"Replication, recombination and repair",NaN,NaN,CP008706.1,AKA29803.1
ABUW_0004,gyrB,NaN,3888,6356,+,DNA gyrase%2C B subunit,"Replication, recombination and repair",NaN,NaN,CP008706.1,AKA29804.1
ABUW_0005,ABUW_0005,NaN,6394,6786,+,cytochrome b562,Energy production and conversion,NaN,NaN,CP008706.1,AKA29805.1
...,...,...,...,...,...,...,...,...,...,...,...
ABUW_5013,ABUW_5013,NaN,7794,8048,-,hypothetical protein,No COG annotation,NaN,NaN,CP008708.1,AKA33691.1
ABUW_5014,ABUW_5014,NaN,8170,8508,-,hypothetical protein,No COG annotation,NaN,NaN,CP008708.1,AKA33692.1
ABUW_6001,ABUW_6001,NaN,1,1182,+,replication protein,No COG annotation,NaN,NaN,CP008709.1,AKA33693.1
ABUW_6002,ABUW_6002,NaN,1185,1388,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33694.1


## Adding new AB5075 locus_tags and gene_names

In [64]:
#Import the new gff files for AB5075 and plasmids
gff_AB5075_new = '../data/external/sequence_files/AB5075_new.gff3'
gff_p1AB5075_new = '../data/external/sequence_files/p1Ab5075_new.gff3'
gff_p2AB5075_new = '../data/external/sequence_files/p2Ab5075_new.gff3'
gff_p3AB5075_new = '../data/external/sequence_files/p3Ab5075_new.gff3'

In [65]:
#Convert gff files to dataframes
DF_annot2 = gff2pandas(gff_AB5075_new,index='locus_tag')
DF_annot2p1 = gff2pandas(gff_p1AB5075_new,index='locus_tag')
DF_annot2p2 = gff2pandas(gff_p2AB5075_new,index='locus_tag')
DF_annot2p3 = gff2pandas(gff_p3AB5075_new,index='locus_tag')

#combine all gff dataframes into one dataframe
DF_annot2 = DF_annot2.append(DF_annot2p1)
DF_annot2 = DF_annot2.append(DF_annot2p2)
DF_annot2 = DF_annot2.append(DF_annot2p3)

DF_annot2 = DF_annot2.reset_index() 
DF_annot2 = DF_annot2.set_index('old_locus_tag') 

DF_annot2.head()

,locus_tag,accession,source,feature,start,end,score,strand,phase,attributes,gene_name,gene_product,ncbi_protein
old_locus_tag,,,,,,,,,,,,,
ABUW_0001,ABUW_RS00005,NZ_CP008706.1,Protein Homology,CDS,95,1492,.,+,0,ID=cds-WP_000964768.1;Parent=gene-ABUW_RS00005...,dnaA,chromosomal replication initiator protein DnaA,WP_000964768.1
ABUW_0002,ABUW_RS00010,NZ_CP008706.1,Protein Homology,CDS,1590,2738,.,+,0,ID=cds-WP_001237350.1;Parent=gene-ABUW_RS00010...,dnaN,DNA polymerase III subunit beta,WP_001237350.1
ABUW_0003,ABUW_RS00015,NZ_CP008706.1,Protein Homology,CDS,2753,3835,.,+,0,ID=cds-WP_000550807.1;Parent=gene-ABUW_RS00015...,recF,DNA replication/repair protein RecF,WP_000550807.1
ABUW_0004,ABUW_RS00020,NZ_CP008706.1,Protein Homology,CDS,3888,6356,.,+,0,ID=cds-WP_000093732.1;Parent=gene-ABUW_RS00020...,gyrB,DNA topoisomerase (ATP-hydrolyzing) subunit B,WP_000093732.1
ABUW_0005,ABUW_RS00025,NZ_CP008706.1,Protein Homology,CDS,6394,6786,.,+,0,ID=cds-WP_000987632.1;Parent=gene-ABUW_RS00025...,cybC,cytochrome b562,WP_000987632.1


In [66]:
#Join common genes between new and old annotation with gene_table
DF_annot2 = DF_annot2.rename(columns={"locus_tag": "new_locus_tag", "accession": "new_accession"})

In [67]:
common = DF_annot2.loc[ica_data.gene_table.index.intersection(DF_annot2.index)]
keep_cols2 = ['new_locus_tag','new_accession']
common = common[keep_cols2]

In [68]:
ica_data.gene_table

,gene_name,old_locus_tag,start,end,strand,gene_product,COG,uniprot,regulator,accession,ncbi_protein
ABUW_6001,ABUW_6001,NaN,1,1182,+,replication protein,No COG annotation,NaN,NaN,CP008709.1,AKA33693.1
ABUW_6002,ABUW_6002,NaN,1185,1388,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33694.1
ABUW_6003,ABUW_6003,NaN,1642,1893,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33695.1
ABUW_5001,repAci1,NaN,1,951,+,DNA replication protein,"Replication, recombination and repair",NaN,NaN,CP008708.1,AKA33680.1
ABUW_5002,ABUW_5002,NaN,944,1519,+,DNA replication protein,No COG annotation,NaN,AbaM,CP008708.1,AKA33681.1
...,...,...,...,...,...,...,...,...,...,...,...
ABUW_4118,ABUW_4118,NaN,79719,81011,+,RumB,"Replication, recombination and repair",NaN,NaN,CP008707.1,AKA33675.1
ABUW_4120,ABUW_4120,NaN,81285,82325,+,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33676.1
ABUW_4121,ABUW_4121,NaN,82570,82833,-,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33677.1
ABUW_4122,ABUW_4122,NaN,82889,83122,-,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33678.1


In [69]:
ica_data.gene_table = ica_data.gene_table.join(common)

In [70]:
ica_data.gene_table

,gene_name,old_locus_tag,start,end,strand,gene_product,COG,uniprot,regulator,accession,ncbi_protein,new_locus_tag,new_accession
ABUW_6001,ABUW_6001,NaN,1,1182,+,replication protein,No COG annotation,NaN,NaN,CP008709.1,AKA33693.1,ABUW_RS19635,NZ_CP008709.1
ABUW_6002,ABUW_6002,NaN,1185,1388,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33694.1,NaN,NaN
ABUW_6003,ABUW_6003,NaN,1642,1893,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33695.1,ABUW_RS19645,NZ_CP008709.1
ABUW_5001,repAci1,NaN,1,951,+,DNA replication protein,"Replication, recombination and repair",NaN,NaN,CP008708.1,AKA33680.1,ABUW_RS19590,NZ_CP008708.1
ABUW_5002,ABUW_5002,NaN,944,1519,+,DNA replication protein,No COG annotation,NaN,AbaM,CP008708.1,AKA33681.1,ABUW_RS19595,NZ_CP008708.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ABUW_4118,ABUW_4118,NaN,79719,81011,+,RumB,"Replication, recombination and repair",NaN,NaN,CP008707.1,AKA33675.1,ABUW_RS19570,NZ_CP008707.1
ABUW_4120,ABUW_4120,NaN,81285,82325,+,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33676.1,ABUW_RS19575,NZ_CP008707.1
ABUW_4121,ABUW_4121,NaN,82570,82833,-,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33677.1,ABUW_RS19580,NZ_CP008707.1
ABUW_4122,ABUW_4122,NaN,82889,83122,-,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33678.1,ABUW_RS19585,NZ_CP008707.1


## Operon details from Biocyc 

In [71]:
biocyc_file = '../data/external/biocyc_file.txt'

DF_biocyc = pd.read_csv(biocyc_file,sep='\t')

# Remove genes with no accession
DF_biocyc = DF_biocyc[DF_biocyc['Accession-1'].notnull()]
DF_biocyc = DF_biocyc[DF_biocyc['Accession-2'].notnull()]

# Set the accession (i.e. locus tag) as index
DF_biocyc = DF_biocyc.set_index('Accession-2')
DF_biocyc = DF_biocyc.sort_values('Accession-2')

# Only keep genes in the final annotation file
DF_biocyc = DF_biocyc.reindex(DF_annot.index)

# Reformat transcription units
DF_biocyc['operon_list'] = DF_biocyc['Genes in same transcription unit'].apply(reformat_biocyc_tu)

In [72]:
DF_biocyc

,All-Genes,Genes in same transcription unit,Accession-1,operon_list
locus_tag,,,,
ABUW_0001,dnaA,dnaA,ABUW_RS00005,dnaA
ABUW_0002,ABUW_RS00010,recF // ABUW_RS00010,ABUW_RS00010,ABUW_RS00010;recF
ABUW_0003,recF,recF // ABUW_RS00010,ABUW_RS00015,ABUW_RS00010;recF
ABUW_0004,gyrB,cybC // gyrB,ABUW_RS00020,cybC;gyrB
ABUW_0005,cybC,cybC // gyrB,ABUW_RS00025,cybC;gyrB
...,...,...,...,...
ABUW_5013,NaN,NaN,NaN,None
ABUW_5014,NaN,NaN,NaN,None
ABUW_6001,ABUW_RS19635,NaN,ABUW_RS19635,None


In [73]:
#Assign unique IDs to operons

# Get all operons
operons = DF_biocyc['operon_list'].unique()

# Map each operon to a unique string
operon_dict = {operon: "Op"+str(i) for i, operon in enumerate(operons)}

# Add names to dataframe
DF_biocyc['operon'] = [operon_dict[op] for op in DF_biocyc["operon_list"]]

DF_biocyc.head()

,All-Genes,Genes in same transcription unit,Accession-1,operon_list,operon
locus_tag,,,,,
ABUW_0001,dnaA,dnaA,ABUW_RS00005,dnaA,Op0
ABUW_0002,ABUW_RS00010,recF // ABUW_RS00010,ABUW_RS00010,ABUW_RS00010;recF,Op1
ABUW_0003,recF,recF // ABUW_RS00010,ABUW_RS00015,ABUW_RS00010;recF,Op1
ABUW_0004,gyrB,cybC // gyrB,ABUW_RS00020,cybC;gyrB,Op2
ABUW_0005,cybC,cybC // gyrB,ABUW_RS00025,cybC;gyrB,Op2


In [74]:
import numpy as np
DF_biocyc['operon'].replace('Op13', np.NaN, inplace=True)

In [75]:
keep_cols3 = ['operon']
DF_biocyc = DF_biocyc[keep_cols3]

In [76]:
DF_biocyc

,operon
locus_tag,
ABUW_0001,Op0
ABUW_0002,Op1
ABUW_0003,Op1
ABUW_0004,Op2
ABUW_0005,Op2
...,...
ABUW_5013,NaN
ABUW_5014,NaN
ABUW_6001,NaN


In [77]:
ica_data.gene_table = ica_data.gene_table.join(DF_biocyc)

In [78]:
ica_data.gene_table.rename(columns = {'end':'stop'}, inplace = True)

In [79]:
ica_data.gene_table

,gene_name,old_locus_tag,start,stop,strand,gene_product,COG,uniprot,regulator,accession,ncbi_protein,new_locus_tag,new_accession,operon
ABUW_6001,ABUW_6001,NaN,1,1182,+,replication protein,No COG annotation,NaN,NaN,CP008709.1,AKA33693.1,ABUW_RS19635,NZ_CP008709.1,NaN
ABUW_6002,ABUW_6002,NaN,1185,1388,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33694.1,NaN,NaN,NaN
ABUW_6003,ABUW_6003,NaN,1642,1893,+,hypothetical protein,No COG annotation,NaN,NaN,CP008709.1,AKA33695.1,ABUW_RS19645,NZ_CP008709.1,NaN
ABUW_5001,repAci1,NaN,1,951,+,DNA replication protein,"Replication, recombination and repair",NaN,NaN,CP008708.1,AKA33680.1,ABUW_RS19590,NZ_CP008708.1,NaN
ABUW_5002,ABUW_5002,NaN,944,1519,+,DNA replication protein,No COG annotation,NaN,AbaM,CP008708.1,AKA33681.1,ABUW_RS19595,NZ_CP008708.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ABUW_4118,ABUW_4118,NaN,79719,81011,+,RumB,"Replication, recombination and repair",NaN,NaN,CP008707.1,AKA33675.1,ABUW_RS19570,NZ_CP008707.1,NaN
ABUW_4120,ABUW_4120,NaN,81285,82325,+,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33676.1,ABUW_RS19575,NZ_CP008707.1,NaN
ABUW_4121,ABUW_4121,NaN,82570,82833,-,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33677.1,ABUW_RS19580,NZ_CP008707.1,NaN
ABUW_4122,ABUW_4122,NaN,82889,83122,-,hypothetical protein,No COG annotation,NaN,NaN,CP008707.1,AKA33678.1,ABUW_RS19585,NZ_CP008707.1,NaN


In [80]:
from pymodulon.io import *

In [81]:
save_to_json(ica_data,'../data/processed_data/abaum_ica_data.json')